In [2]:
import vegaapiclient as vac
import math
from utils.risk_models import LogNormal

# testnet
# node_url_grpc="n06.testnet.vega.xyz:3007"
# devnet
# node_url_grpc="n04.d.vega.xyz:3007"
# stagnet2
node_url_grpc="n01.stagnet2.vega.xyz:3007"
data_client = vac.VegaTradingDataClient(node_url_grpc)

# helper methods
def round_sd(number):
    significant_digits = 4
    return round(number, significant_digits - math.floor(math.log10(abs(number))) - 1)

def calculate_exit_price(order_book, position, market_dp):
    levels = order_book.sell
    if position < 0:
        levels = order_book.buy
    p = position
    v = 0
    exit_price = 0
    for lvl in levels:
        x = lvl.volume
        if lvl.volume > p:
            x = p
        exit_price += x*float(lvl.price)*10**(-market_dp)
        v+=x
        p-=x
        if p <= 0:
            break
    return exit_price / v

def calculate_slippage_per_unit(mark_price, order_book, open_volume, market_dp):
    if open_volume==0:
        return 0
    exit_price=calculate_exit_price(order_book,open_volume,market_dp)
    s = 1 if open_volume > 0 else -1
    return s*(mark_price - exit_price)

### Specify public key for the party.
Specify the your public key in the cell below. It can be found in the wallet side panel in the Console.

In [3]:
pubkey="663a860e7b32077a4b3fbf2bc46a28cd36c1a41aa74ab33a177cf4d597e87ce0"

### Print party info

In [7]:
party_positions = data_client.PositionsByParty(vac.data_node.api.v1.trading_data.PositionsByPartyRequest(party_id=pubkey))
if len(party_positions.positions) == 0:
    print("party with the public key specified has no open positions")
    quit(keep_kernel=True)
party_accounts = data_client.PartyAccounts(vac.data_node.api.v1.trading_data.PartyAccountsRequest(party_id=pubkey))

for pos in party_positions.positions:
    market=data_client.MarketByID(vac.data_node.api.v1.trading_data.MarketByIDRequest(market_id=pos.market_id)).market
    market_name=market.tradable_instrument.instrument.name
    market_data=data_client.MarketDataByID(vac.data_node.api.v1.trading_data.MarketDataByIDRequest(market_id=pos.market_id)).market_data
    margin_levels=data_client.MarginLevels(vac.data_node.api.v1.trading_data.MarginLevelsRequest(party_id=pubkey, market_id=pos.market_id)).margin_levels
  
    margin_acc = next(x for x in party_accounts.accounts if x.market_id == pos.market_id )
    gen_acc = next(x for x in party_accounts.accounts if x.type == vac.vega.vega.ACCOUNT_TYPE_GENERAL and x.asset==margin_acc.asset)
    
    asset_dp = data_client.AssetByID(vac.data_node.api.v1.trading_data.AssetByIDRequest(id=gen_acc.asset)).asset.details.decimals
    market_dp = market.decimal_places
    
    mar_acc_bal = float(margin_acc.balance)*10**(-asset_dp)
    gen_acc_bal = float(gen_acc.balance)*10**(-asset_dp)
    
    notional=abs(pos.open_volume)*float(market_data.mark_price)*10**(-market_dp)
    current_leverage=round_sd(notional/(gen_acc_bal+mar_acc_bal))
    capital_utilisation=round_sd((mar_acc_bal/(mar_acc_bal+gen_acc_bal))*100)
    
    mar_mant = float(margin_levels[0].maintenance_margin)*10**(-market_dp)
    mar_srch = float(margin_levels[0].search_level)*10**(-market_dp)
    mar_init = float(margin_levels[0].initial_margin)*10**(-market_dp)
    mar_rel = float(margin_levels[0].collateral_release_level)*10**(-market_dp)
   
    mark_price=float(market_data.mark_price)*10**(-market_dp)
    
    search_price_first_approx = (mar_srch - mar_acc_bal)/pos.open_volume + mark_price
    liquidation_price_first_approx = (mar_mant - mar_acc_bal - gen_acc_bal)/pos.open_volume + mark_price

    p=market.tradable_instrument.log_normal_risk_model
    risk_model = LogNormal(mu=p.params.mu,sigma=p.params.sigma, lambd=p.risk_aversion_parameter, tau=p.tau)
    risk_factor = risk_model.RiskFactorLong() if pos.open_volume >= 0 else risk_model.RiskFactorShort()
    scaling_factors = market.tradable_instrument.margin_calculator.scaling_factors

    market_depth = data_client.MarketDepth(vac.data_node.api.v1.trading_data.MarketDepthRequest(market_id=market.id, max_depth=abs(pos.open_volume)))
    slippage_per_unit = calculate_slippage_per_unit(mark_price,market_depth, pos.open_volume, market_dp)

    c_1 = max(abs(pos.open_volume)*slippage_per_unit,0)
    c_2 = pos.open_volume * risk_factor

    search_price_second_approx = (mar_acc_bal -  pos.open_volume*mark_price-scaling_factors.search_level *c_1 )/(scaling_factors.search_level*c_2-pos.open_volume)
    liquidation_price_second_approx = (mar_acc_bal + gen_acc_bal - pos.open_volume*mark_price-c_1 )/(c_2-pos.open_volume)

    print("\tmarket: {name}".format(name=market_name)) 
    print("\n")
#     market info
    print("\t\t{:<29} {}".format("mark price:",round_sd(mark_price)))
#     party info
    print("\t\t{:<29} {}".format("party position:",pos.open_volume))
    print("\t\t{:<29} {}".format("margin account:",round_sd(mar_acc_bal)))
    print("\t\t{:<29} {}".format("general account:",round_sd(gen_acc_bal)))
    print("\n")
#     leverage
    print("\t\t{:<28} ~{}%".format("capital utilisation:", capital_utilisation*100))
    print("\t\t{:<28} ~{}x".format("effective account leverage:", current_leverage))
    print("\t\t{:<28} ~{}x".format("theoretical max leverage:", round_sd(1/risk_factor)))
    print("\t\tleverage per margin level:")
    print("\t\t\t{:<20} ~{}x".format("maintenance:",round_sd(notional/mar_mant)))
    print("\t\t\t{:<20} ~{}x".format("search",round_sd(notional/mar_srch)))
    print("\t\t\t{:<20} ~{}x".format("initial",round_sd(notional/mar_init)))
    print("\t\t\t{:<20} ~{}x".format("release",round_sd(notional/mar_rel)))
    print("\n")
#     margin levels
    print("\t\tmargin levels:")
    print("\t\t\t{:<20} ~{}".format("search:",mar_srch))
    print("\t\t\t{:<20} ~{}".format("maintenance:",mar_mant))
#     price approximation of margin levels
    print("\t\tfirst approximation price:")
    print("\t\t\t{:<20} ~{}".format("search:",max(0,round_sd(search_price_first_approx))))
    print("\t\t\t{:<20} ~{}".format("liquidation:",max(0,round_sd(liquidation_price_first_approx))))
    print("\t\tsecond approximation price:")
    print("\t\t\t{:<20} ~{}".format("search:",max(0,round_sd(search_price_second_approx))))
    print("\t\t\t{:<20} ~{}".format("liquidation:",max(0,round_sd(liquidation_price_second_approx))))
    print("\n\n")
    

	market: UNIDAI Monthly (30 Jun 2022)


		mark price:                   5.208
		party position:               -486
		margin account:               176.9
		general account:              10500000.0


		capital utilisation:         ~0.1685%
		effective account leverage:  ~0.0002411x
		theoretical max leverage:    ~69.82x
		leverage per margin level:
			maintenance:         ~21.46x
			search               ~19.51x
			initial              ~14.31x
			release              ~12.62x


		margin levels:
			search:              ~129.74525
			maintenance:         ~117.95023
		first approximation price:
			search:              ~5.305
			liquidation:         ~21610.0
		second approximation price:
			search:              ~5.492
			liquidation:         ~21920.0



